## 8 Contraintes de rampe

### 8.1 rampe croissante

In [ ]:
CENTRALE, nb, Pmin, Pmax, Cstart, Cbase, Cmwh, rpc, rpd  = gp.multidict({
    'A' : [12, 850, 2000, 2000, 1000, 2.0, 400, 1000],
    'B' : [10, 1250, 1750, 1000, 2600, 1.3, 600, 1500],
    'C' : [5, 1500, 4000, 500, 3000, 3, 800, 2000],
})

HYDRO, pfix, vab, cdem, cfonc = gp.multidict({
    '1' : [gp.tupledict([(0,900), (1,950),(2,1000), (3,1100)]), gp.tupledict([(0,0.31), (1,0.33),(2,0.35), (3,0.38)]), 1500, gp.tupledict([(0,90), (1,95),(2,105), (3,120)])],
    '2' : [gp.tupledict([(0,1400), (1,1500),(2,1600), (3,1700)]),gp.tupledict([(0,0.47), (1,0.50),(2,0.53), (3,0.56)]), 1200, gp.tupledict([(0,150), (1,165),(2,185), (3,210)])],
})

Dem = [15, 15, 15, 15, 15, 15, 30 , 30 , 30, 25, 25, 25, 25, 25, 25, 40, 40, 40, 27, 27, 27, 27, 27, 27]

for i in range(len(Dem)):
    Dem[i]=Dem[i]*10**3
    
Reserve = 1.15

Model = gp.Model(name = 'Centrales thermiques2')

NBcentrale = {(i,j) : Model.addVar(vtype=gp.GRB.INTEGER,
                                  lb=0,  ub=nb[i])                             
              for i in CENTRALE for j in range(len(Dem))}

Puiss = {(i,j,k) : Model.addVar(vtype=gp.GRB.CONTINUOUS)
                for i in CENTRALE for j in range(len(Dem)) for k in range(nb[i])}

Use = {(i,j,k) : Model.addVar(vtype=gp.GRB.BINARY)
                for i in CENTRALE for j in range(len(Dem)) for k in range(nb[i])}

NBstart = {(i,j) : Model.addVar(vtype=gp.GRB.CONTINUOUS,
                               lb=0)
                for i in CENTRALE for j in range(len(Dem))}

Usehydro = {(h,p,j) : Model.addVar(vtype=gp.GRB.BINARY,
                               lb=0, ub=1)
                for h in HYDRO for p in range(len(pfix['1'])) for j in range(len(Dem))}

hydrostart = {(h,j) : Model.addVar(vtype=gp.GRB.BINARY,
                               lb=0, ub=1)
                for h in HYDRO for j in range(len(Dem))}

Startpump = { j : Model.addVar(vtype=gp.GRB.CONTINUOUS,
                               lb=0, ub=1)
                for j in range(len(Dem))}

POT = {j: sum(NBcentrale[i,j]*Pmax[i] for i in CENTRALE) + sum((1-sum(Usehydro[h,p,j] for p in vab['1']))*pfix[h][3] for h in HYDRO)
       for j in range(len(Dem))}            

CSTR1 = { (i,j,k) : Model.addLConstr(Pmin[i] * Use[i,j,k] , "<=" ,Puiss[i,j,k])
         for i in CENTRALE for j in range(len(Dem)) for k in range(nb[i])}

CSTR2 = { (i,j,k) : Model.addLConstr(Pmax[i] * Use[i,j,k], ">=" ,Puiss[i,j,k])
         for i in CENTRALE for j in range(len(Dem)) for k in range(nb[i])}

CSTR3 = { j : Model.addLConstr(sum(Puiss[i,j,k] for i in CENTRALE for k in range(nb[i])) + sum(Usehydro[h,p,j]*pfix[h][p] for h in HYDRO for p in vab[h]),"=",Dem[j] + Startpump[j] * 3000)
         for j in range(len(Dem))}
         
CSTR4 =  { (i,j) : Model.addLConstr(NBcentrale[i,j],"<=",nb[i])
         for i in CENTRALE for j in range(len(Dem))}

CSTR5 =  { (i,j) : Model.addLConstr(NBstart[i,j],">=",NBcentrale[i,j]-NBcentrale[i,j-1])
         for i in CENTRALE for j in range(1,len(Dem))}

CSTR_41 =  { (i) : Model.addLConstr(NBstart[i,0],">=",NBcentrale[i,0]- NBcentrale[i,23])
         for i in CENTRALE }

CSTR_31 =  { j : Model.addLConstr(POT[j],">=",Dem[j]*Reserve)
             for j in range(len(Dem))}

CSTR_511 = {(h,j) : Model.addLConstr(hydrostart[h,j],">=",sum(Usehydro[h,p,j] - Usehydro[h,p,j-1] for p in vab[h]))
             for h in HYDRO for j in range(1,len(Dem))}

CSTR_512 = { h : Model.addLConstr(hydrostart[h,0],">=",sum(Usehydro[h,p,0] - Usehydro[h,p,23] for p in vab[h]))
             for h in HYDRO}

Cnew = {(i,j): NBcentrale[i,j]*Cbase[i] + (sum(Puiss[i,j,k] for k in range(nb[i])) - NBcentrale[i,j] * Pmin[i]) * Cmwh[i] 
       for i in CENTRALE for j in range(len(Dem))}

Conseau = sum(vab[h][p] * Usehydro[h,p,j] for h in HYDRO for p in vab[h] for j in range(len(Dem)))

CSTR_52 = {Model.addLConstr(sum(Startpump[j] for j in range(len(Dem))),"=",Conseau)}

CSTR_53 = {(h,j) : Model.addLConstr(sum(Usehydro[h,p,j] for p in vab[h]),"<=",1)
          for h in HYDRO for j in range(len(Dem))}

CSTR_54 = { j : Model.addLConstr(Startpump[j],"<=",1-sum(Usehydro['2',p,j] for p in vab['2']))
          for j in range(len(Dem))}

CSTR_54 = { j : Model.addLConstr(Startpump[j],"<=",1-sum(Usehydro['1',p,j] for p in vab['1']))
          for j in range(len(Dem))}

CSTR_611 = { (i,j) : Model.addLConstr(sum(Use[i,j,k] for k in range(nb[i])),"=",NBcentrale[i,j])
          for i in CENTRALE for j in range(len(Dem))}

CSTR_811 =  { (i,j,k) : Model.addLConstr(Puiss[i,j,k],"<=",Puiss[i,j-1,k] + rpc[i])
         for i in CENTRALE for j in range(1,len(Dem)) for k in range(nb[i])}

CSTR_812 =  { (i) : Model.addLConstr(Puiss[i,0,k],">=",Puiss[i,23,k] + rpc[i])
         for i in CENTRALE for k in range(nb[i])}
                                  
Model.setObjective(sum(Cnew[i,j] + NBstart[i,j] * Cstart[i]  for i in CENTRALE for j in range(len(Dem))) + sum(Usehydro[h,p,j] * cfonc[h][p] for h in HYDRO for p in vab[h] for j in range(len(Dem))) + sum(hydrostart[h,j]*cdem[h] for h in HYDRO for j in range(len(Dem))),gp.GRB.MINIMIZE)